In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import json
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.pyplot import figure
import skimage.transform
import argparse
from PIL import Image


from tqdm import tqdm
from os import listdir

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/opt/homebrew/Caskroom/miniforge/base/envs/kiki/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [2]:
def caption_image_beam_search(encoder, decoder, image_path, word_map, beam_size=3):
    
    rev_map = {v:k for k,v in word_map.items()}
    
    """
    Reads an image and captions it with beam search.

    :param encoder: encoder model
    :param decoder: decoder model
    :param image_path: path to image
    :param word_map: word map
    :param beam_size: number of sequences to consider at each decode-step
    :return: caption, weights for visualization
    """

    k = beam_size
    vocab_size = len(word_map)
    
    
    img = Image.open(image_path)
    img = img.resize((256, 256))
    img = np.transpose(img, (2, 0, 1))
    #img = img / 255.
    img = torch.FloatTensor(img).to(device)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([normalize])
    image = transform(img)  # (3, 256, 256)

    # Encode
    image = image.unsqueeze(0)  # (1, 3, 256, 256)
    encoder_out = encoder(image)  # (1, enc_image_size, enc_image_size, encoder_dim)
    enc_image_size = encoder_out.size(1)
    encoder_dim = encoder_out.size(3)

    # Flatten encoding
    encoder_out = encoder_out.view(1, -1, encoder_dim)  # (1, num_pixels, encoder_dim)
    num_pixels = encoder_out.size(1)

    # We'll treat the problem as having a batch size of k
    encoder_out = encoder_out.expand(k, num_pixels, encoder_dim)  # (k, num_pixels, encoder_dim)

    # Tensor to store top k previous words at each step; now they're just <start>
    k_prev_words = torch.LongTensor([[word_map['<start>']]] * k).to(device)  # (k, 1)

    # Tensor to store top k sequences; now they're just <start>
    seqs = k_prev_words  # (k, 1)

    # Tensor to store top k sequences' scores; now they're just 0
    top_k_scores = torch.zeros(k, 1).to(device)  # (k, 1)

    # Tensor to store top k sequences' alphas; now they're just 1s
    seqs_alpha = torch.ones(k, 1, enc_image_size, enc_image_size).to(device)  # (k, 1, enc_image_size, enc_image_size)

    # Lists to store completed sequences, their alphas and scores
    complete_seqs = list()
    complete_seqs_alpha = list()
    complete_seqs_scores = list()
    itera = 0
    # Start decoding
    step = 1
    h, c = decoder.init_hidden_state(encoder_out)

    # s is a number less than or equal to k, because sequences are removed from this process once they hit <end>
    while True:

        embeddings = decoder.embedding(k_prev_words).squeeze(1)  # (s, embed_dim)

        awe, alpha = decoder.attention(encoder_out, h)  # (s, encoder_dim), (s, num_pixels)

        alpha = alpha.view(-1, enc_image_size, enc_image_size)  # (s, enc_image_size, enc_image_size)

        gate = decoder.sigmoid(decoder.f_beta(h))  # gating scalar, (s, encoder_dim)
        awe = gate * awe

        h, c = decoder.decode_step(torch.cat([embeddings, awe], dim=1), (h, c))  # (s, decoder_dim)

        scores = decoder.fc(h)  # (s, vocab_size)
        scores = F.log_softmax(scores, dim=1)

        # Add
        scores = top_k_scores.expand_as(scores) + scores  # (s, vocab_size)

        # For the first step, all k points will have the same scores (since same k previous words, h, c)
        if step == 1:
            top_k_scores, top_k_words = scores[0].topk(k, 0, True, True)  # (s)
        else:
            # Unroll and find top scores, and their unrolled indices
            top_k_scores, top_k_words = scores.view(-1).topk(k, 0, True, True)  # (s)

        # Convert unrolled indices to actual indices of scores
        prev_word_inds = top_k_words / vocab_size  # (s)
        next_word_inds = top_k_words % vocab_size  # (s)
        
        
        # Add new words to sequences, alphas

        prev_word_inds = prev_word_inds.cpu()
        prev_word_inds = prev_word_inds.long()        

        seqs = torch.cat([seqs[prev_word_inds], next_word_inds.unsqueeze(1)], dim=1)  # (s, step+1)
        
        
        witers = list()
        for row in seqs:
            for item in row:
                witers.append(rev_map[int(item)])
      
        itera += 1
#         print(itera)
        
#         print(witers)
        
        seqs_alpha = torch.cat([seqs_alpha[prev_word_inds], alpha[prev_word_inds].unsqueeze(1)],
                               dim=1)  # (s, step+1, enc_image_size, enc_image_size)

        # Which sequences are incomplete (didn't reach <end>)?
        incomplete_inds = [ind for ind, next_word in enumerate(next_word_inds) if
                           next_word != word_map['<end>']]
        complete_inds = list(set(range(len(next_word_inds))) - set(incomplete_inds))

        # Set aside complete sequences
        if len(complete_inds) > 0:
            complete_seqs.extend(seqs[complete_inds].tolist())
            complete_seqs_alpha.extend(seqs_alpha[complete_inds].tolist())
            complete_seqs_scores.extend(top_k_scores[complete_inds])
        k -= len(complete_inds)  # reduce beam length accordingly

        # Proceed with incomplete sequences
        if k == 0:
            break
        seqs = seqs[incomplete_inds]
        seqs_alpha = seqs_alpha[incomplete_inds]
        h = h[prev_word_inds[incomplete_inds]]
        c = c[prev_word_inds[incomplete_inds]]
        encoder_out = encoder_out[prev_word_inds[incomplete_inds]]
        top_k_scores = top_k_scores[incomplete_inds].unsqueeze(1)
        k_prev_words = next_word_inds[incomplete_inds].unsqueeze(1)

        # Break if things have been going on too long
        if step > 50:
            break
        step += 1

#     i = complete_seqs_scores.index(max(complete_seqs_scores))
    try:
        seq = complete_seqs[-1]
        alphas = complete_seqs_alpha[-1]
    except:
        seq = complete_seqs
        alphas = complete_seqs_alpha

    return seq, alphas


def visualize_att(image_path, seq, alphas, rev_word_map, smooth=False):
    """
    Visualizes caption with weights at every word.

    Adapted from paper authors' repo: https://github.com/kelvinxu/arctic-captions/blob/master/alpha_visualization.ipynb

    :param image_path: path to image that has been captioned
    :param seq: caption
    :param alphas: weights
    :param rev_word_map: reverse word mapping, i.e. ix2word
    :param smooth: smooth weights?
    """
    
    figure(figsize=(10, 8), dpi=80)
    
    image = Image.open(image_path)
    image = image.resize([24 * 38, 24 * 38], Image.LANCZOS)

    words = [rev_word_map[ind] for ind in seq]

    for t in range(len(words)):
        if t > 40:
            break
        
        rows = int(np.ceil(len(words) / 5))
#         print(rows)

        plt.subplot(rows, 5, t + 1)

        plt.text(0, 1, '%s' % (words[t]), color='black', backgroundcolor='white', fontsize=12)
        plt.imshow(image)
        current_alpha = alphas[t, :]
        if smooth:
            alpha = skimage.transform.pyramid_expand(current_alpha.numpy(), upscale=12, sigma=8)
        else:
            alpha = skimage.transform.resize(current_alpha.numpy(), [24 * 38, 24 * 38])
        if t == 0:
            plt.imshow(alpha, alpha=0)
        else:
            plt.imshow(alpha, alpha=0.6)
        plt.set_cmap(cm.Greys_r)
        plt.axis('off')
    plt.show()

    


In [3]:
def generate_predictions(model_path, prediciton_file, word_map, data_path, beam_size, smooth):
    # Load word map (word2ix)
    with open(word_map, 'r') as j:
        word_map = json.load(j)
    rev_word_map = {v: k for k, v in word_map.items()}  # ix2word
            
    predictions = dict()
    
    for f_name in tqdm(listdir(data_path)):
        img_id = int(f_name.split('.')[0])
        img = data_path +f_name
        
        if 9000 <= img_id < 10000:

            # Load model
            checkpoint = torch.load(model_path, map_location=str(device))
            decoder = checkpoint['decoder']
            decoder = decoder.to(device)
            decoder.eval()
            encoder = checkpoint['encoder']
            encoder = encoder.to(device)
            encoder.eval()

            # Encode, decode with attention and beam search
            seq, alphas = caption_image_beam_search(encoder, decoder, img, word_map, beam_size)
            alphas = torch.FloatTensor(alphas)

            words = [rev_word_map[ind] for ind in seq]

            predictions[img_id] = words
    
    with open(out_path+'/'+prediciton_file, mode='w') as f:
        json.dump(predictions, f)

In [10]:
model = 'm3'
trained_on = 'imgs'
predictions_on = 'freakshow'


training_preprocessed_data = 'out7'

In [16]:
model_path = f'/Users/evelsve/repos/cap/models/{model}/BEST_checkpoint_celeb_small_res_5_10.pth.tar'
word_map = f'/Users/evelsve/repos/cap/output/{training_preprocessed_data}/wordmap_celeb_small_res_5_10.json'

data_path = f'/Users/evelsve/repos/cap/data/{predictions_on}/img/'

out_path = '/Users/evelsve/repos/cap/predictions'

prediction_file = f'predictions_{predictions_on}_{model}_trained_on_{trained_on}.json'


beam_size = 3
smooth = False
generate_predictions(model_path, prediction_file, word_map, data_path, beam_size, smooth)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [07:08<00:00, 23.34it/s]


In [ ]:
# model_path = '/Users/evelsve/repos/cap/m5/BEST_checkpoint_celeb_small_res_5_10.pth.tar'
# # model_path = '/Users/evelsve/repos/cap/old/BEST_checkpoint_flickr8k_5_10.pth.tar'
# img = '/Users/evelsve/repos/cap/data/celeb_small_res/img/41.jpg'
# # img = '/Users/evelsve/repos/cap/data/sketches/1.jpg'
# # img = '/Users/evelsve/repos/cap/flickr8k/Images/1287982439_6578006e22.jpg'

# # 1346529555_e916816cfe
# # 1461329041_c623b06e5b
# # 124972799_de706b6d0b
# # 1287982439_6578006e22

# # word_map = '/Users/evelsve/repos/cap/flickr8k/wordmap_flickr8k_5_10.json'
# word_map = '/Users/evelsve/repos/cap/out9/wordmap_celeb_small_res_5_10.json'
# beam_size = 4
# smooth = False

# # Load model
# checkpoint = torch.load(model_path, map_location=str(device))
# decoder = checkpoint['decoder']
# decoder = decoder.to(device)
# decoder.eval()
# encoder = checkpoint['encoder']
# encoder = encoder.to(device)
# encoder.eval()

# # Load word map (word2ix)
# with open(word_map, 'r') as j:
#     word_map = json.load(j)
# rev_word_map = {v: k for k, v in word_map.items()}  # ix2word

# # Encode, decode with attention and beam search
# seq, alphas = caption_image_beam_search(encoder, decoder, img, word_map, beam_size)
# alphas = torch.FloatTensor(alphas)

# # Visualize caption and attention of best sequence
# visualize_att(img, seq, alphas, rev_word_map, smooth)